In [ ]:
def model_fn(model_path): #Loading model
    model = load_model('model.hdf5')
    return model

def input_fn(input_data):
    try:
        with open(input_data) as f:
            data = json.load(f)
    except:
        print("There's no file")
    else:
        columns = ['nr_measurement', 'time', 'nr_sensor', 'pressure', 'position', 'mode']
        df = pd.DataFrame(columns = columns)
        time = datetime.now(pytz.timezone('Europe/Warsaw')).strftime("%Y-%m-%d %H:%M:%S")
        frame1 = [i,time, "0", round(data["output"]['0']['pressure'],2), data["output"]['0']['position'], data["output"]['0']['mode'] ]
        frame2 = [i,time, "1", round(data["output"]['1']['pressure'],2), data["output"]['1']['position'], data["output"]['1']['mode'] ]
        frame3 = [i,time, "2", round(data["output"]['2']['pressure'],2), data["output"]['2']['position'], data["output"]['2']['mode'] ]
        frame4 = [i,time, "3", round(data["output"]['3']['pressure'],2), data["output"]['3']['position'], data["output"]['3']['mode'] ]     
        frame = pd.DataFrame( [frame1, frame2, frame3, frame4], columns = columns)
        df = df.append(frame, sort=False)
        i+=1
    return df
    
def predict_fn(input_data, model):
    predictions = model.predict(input_data)
    return prediction

    
def output_fn(predicition, num):
    time = datetime.now(pytz.timezone('Europe/Warsaw')).strftime("%Y-%m-%d %H:%M:%S")
    json = {"time": time,
            "RUL": round(predict(),2),   #RUL - real usage life
            "anomaly": random.choice((True, False)),
            "sensor": randint(0, 8) }
    file_name = str(num + ".json")
    s3 = boto3.resource('s3')
    s3.Bucket("neurosys-intern").put_object(Key=file_name, Body=json, ACL="public-read-write")
    
    
if __name__ = 'main':
    parser = argparse.ArgumentParser() 
    # Data, model, and output directories
    parser.add_argument('--output-data-dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
  
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]

    input_fn()
    model_fn()
    predict_fn()
    output_fn()